# **Enfoque en el Análisis de Datos**
El principal objetivo de este desafío es comenzar a desarrollar tus habilidades en análisis de datos con Python. Para que puedas enfocarte en la lógica del análisis y en la extracción de insights, te proporcionamos un cuaderno base en GitHub con las cuatro bases de datos ya preparadas. Así, no tendrás que preocuparte por la extracción de los archivos CSV y podrás dedicarte directamente a la manipulación y visualización de los datos.

## Accede al cuaderno base

[Repositorio Base - Alura Store](https://github.com/alura-es-cursos/challenge1-data-science-latam)

Con este material listo, podrás aplicar conceptos clave como manipulación de datos con Pandas, estructuración de datos en listas, uso de condicionales, funciones y visualización de información para obtener insights valiosos sobre las tiendas de Alura Store.

## **Solución:**

In [12]:
import pandas as pd

"""
url = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"
"""

# 1. Definición de archivos y carga de datos
file_names = {
    'tienda_1 .csv': 'Tienda 1',
    'tienda_2.csv': 'Tienda 2',
    'tienda_3.csv': 'Tienda 3',
    'tienda_4.csv': 'Tienda 4'
}

all_data = []

# Carga de archivos y adición de la columna 'Tienda'
for file_name, store_id in file_names.items():
    df = pd.read_csv(file_name)
    df['Tienda'] = store_id
    all_data.append(df)

In [11]:
df.head()

,Producto,Categoría del Producto,Precio,Costo de envío,Fecha de Compra,Vendedor,Lugar de Compra,Calificación,Método de pago,Cantidad de cuotas,lat,lon,Tienda
0,Lavavajillas,Electrodomésticos,1084800.0,56000.0,11/11/2022,Rafael Acosta,Bogotá,4,Tarjeta de crédito,3,4.60971,-74.08175,Tienda 4
1,Cama box,Muebles,473800.0,25400.0,03/01/2020,Izabela de León,Cúcuta,5,Tarjeta de crédito,2,7.89391,-72.50782,Tienda 4
2,Balón de baloncesto,Deportes y diversión,23700.0,0.0,25/12/2022,Maria Alfonso,Medellín,5,Nequi,1,6.25184,-75.56359,Tienda 4
3,Modelado predictivo,Libros,57700.0,1300.0,23/11/2021,Lucas Olivera,Medellín,3,Nequi,1,6.25184,-75.56359,Tienda 4
4,Auriculares,Electrónicos,138900.0,5600.0,25/04/2022,Blanca Ramirez,Bogotá,4,Tarjeta de crédito,7,4.60971,-74.08175,Tienda 4


In [2]:
# Concatenación y limpieza de datos
df_concentrado = pd.concat(all_data, ignore_index=True)
df_concentrado['Precio'] = pd.to_numeric(df_concentrado['Precio'], errors='coerce')
df_concentrado['Costo de envío'] = pd.to_numeric(df_concentrado['Costo de envío'], errors='coerce')
df_concentrado['Calificación'] = pd.to_numeric(df_concentrado['Calificación'], errors='coerce')
df_concentrado['Fecha de Compra'] = pd.to_datetime(df_concentrado['Fecha de Compra'], format='%d/%m/%Y', errors='coerce')

In [3]:
# 2. Análisis por Tienda (Agrupación)
# 1. Análisis de Facturación, 3. Calificación Promedio, 5. Envío Promedio
facturacion_tienda = df_concentrado.groupby('Tienda')['Precio'].sum().reset_index()
calificacion_tienda = df_concentrado.groupby('Tienda')['Calificación'].mean().reset_index()
envio_tienda = df_concentrado.groupby('Tienda')['Costo de envío'].mean().reset_index()

df_resumen_tienda = facturacion_tienda.merge(calificacion_tienda, on='Tienda').merge(envio_tienda, on='Tienda')
df_resumen_tienda.rename(columns={
    'Precio': 'Facturación Total',
    'Calificación': 'Calificación Promedio',
    'Costo de envío': 'Costo de Envío Promedio'
}, inplace=True)
print("Resumen de Métricas Clave por Tienda:")
print(df_resumen_tienda.to_markdown(index=False, floatfmt=(",.2f", ".2f")))

Resumen de Métricas Clave por Tienda:
| Tienda   |   Facturación Total |   Calificación Promedio |   Costo de Envío Promedio |
|:---------|--------------------:|------------------------:|--------------------------:|
| Tienda 1 |       1150880400.00 |                 3.97669 |                   26018.6 |
| Tienda 2 |       1116343500.00 |                 4.0373  |                   25216.2 |
| Tienda 3 |       1098019600.00 |                 4.04833 |                   24805.7 |
| Tienda 4 |       1038375700.00 |                 3.99576 |                   23459.5 |


In [4]:
# 2. Ventas por Categoría (Top 3 Ingreso)
def top_categories_by_store(df):
    store_results = {}
    for store in df['Tienda'].unique():
        df_store = df[df['Tienda'] == store]
        top_cats = df_store.groupby('Categoría del Producto')['Precio'].sum().nlargest(3).reset_index()
        top_cats.rename(columns={'Precio': 'Ingreso Total'}, inplace=True)
        store_results[store] = top_cats
    return store_results

ventas_categoria_tienda = top_categories_by_store(df_concentrado)
print("\nTop 3 Categorías con Mayor Ingreso por Tienda:")
for store, df_cat in ventas_categoria_tienda.items():
    print(f"\n--- {store} ---")
    print(df_cat.to_markdown(index=False, floatfmt=",.2f"))


Top 3 Categorías con Mayor Ingreso por Tienda:

--- Tienda 1 ---
| Categoría del Producto   |   Ingreso Total |
|:-------------------------|----------------:|
| Electrónicos             |  429,493,500.00 |
| Electrodomésticos        |  363,685,200.00 |
| Muebles                  |  187,633,700.00 |

--- Tienda 2 ---
| Categoría del Producto   |   Ingreso Total |
|:-------------------------|----------------:|
| Electrónicos             |  410,831,100.00 |
| Electrodomésticos        |  348,567,800.00 |
| Muebles                  |  176,426,300.00 |

--- Tienda 3 ---
| Categoría del Producto   |   Ingreso Total |
|:-------------------------|----------------:|
| Electrónicos             |  410,775,800.00 |
| Electrodomésticos        |  329,237,900.00 |
| Muebles                  |  201,072,100.00 |

--- Tienda 4 ---
| Categoría del Producto   |   Ingreso Total |
|:-------------------------|----------------:|
| Electrónicos             |  409,476,100.00 |
| Electrodomésticos        |  283,

In [5]:
# 4. Productos Más y Menos Vendidos (Top/Least 3 Volumen)
def top_least_products_by_store(df):
    store_results = {}
    for store in df['Tienda'].unique():
        df_store = df[df['Tienda'] == store]
        product_counts = df_store['Producto'].value_counts()

        top_3 = product_counts.nlargest(3).reset_index()
        top_3.columns = ['Producto', 'Cantidad Vendida']

        least_3 = product_counts.nsmallest(3).reset_index()
        least_3.columns = ['Producto', 'Cantidad Vendida']

        store_results[store] = {'Mas Vendidos': top_3, 'Menos Vendidos': least_3}
    return store_results

productos_tienda = top_least_products_by_store(df_concentrado)
print("\nTop 3 Productos Más y Menos Vendidos por Tienda:")
for store, results in productos_tienda.items():
    print(f"\n--- {store} ---")
    print("Mas Vendidos:")
    print(results['Mas Vendidos'].to_markdown(index=False))
    print("\nMenos Vendidos:")
    print(results['Menos Vendidos'].to_markdown(index=False))


Top 3 Productos Más y Menos Vendidos por Tienda:

--- Tienda 1 ---
Mas Vendidos:
| Producto      |   Cantidad Vendida |
|:--------------|-------------------:|
| Microondas    |                 60 |
| TV LED UHD 4K |                 60 |
| Armario       |                 60 |

Menos Vendidos:
| Producto                  |   Cantidad Vendida |
|:--------------------------|-------------------:|
| Auriculares con micrófono |                 33 |
| Celular ABXY              |                 33 |
| Olla de presión           |                 35 |

--- Tienda 2 ---
Mas Vendidos:
| Producto                  |   Cantidad Vendida |
|:--------------------------|-------------------:|
| Iniciando en programación |                 65 |
| Microondas                |                 62 |
| Batería                   |                 61 |

Menos Vendidos:
| Producto        |   Cantidad Vendida |
|:----------------|-------------------:|
| Juego de mesa   |                 32 |
| Mesa de comedor |     

In [8]:
# 3. Análisis Global (Resumen Concentrado)
print("\n--- Análisis Global (Resumen Concentrado) ---")

facturacion_global = df_concentrado['Precio'].sum()
calificacion_global = df_concentrado['Calificación'].mean()
envio_global = df_concentrado['Costo de envío'].mean()

print(f"1. Facturación Global Total: ${facturacion_global:,.2f}")
print(f"3. Calificación Promedio Global: {calificacion_global:.2f}")
print(f"5. Costo de Envío Promedio Global: ${envio_global:,.2f}")


--- Análisis Global (Resumen Concentrado) ---
1. Facturación Global Total: $4,403,619,200.00
3. Calificación Promedio Global: 4.01
5. Costo de Envío Promedio Global: $24,875.15


In [9]:
# 2. Top 5 Categorías Globales
ventas_categoria_global = df_concentrado.groupby('Categoría del Producto')['Precio'].sum().nlargest(5).reset_index()
ventas_categoria_global.rename(columns={'Precio': 'Ingreso Total'}, inplace=True)
print("\n2. Top 5 Categorías con Mayor Ingreso Global:")
print(ventas_categoria_global.to_markdown(index=False, floatfmt=",.2f"))


2. Top 5 Categorías con Mayor Ingreso Global:
| Categoría del Producto   |    Ingreso Total |
|:-------------------------|-----------------:|
| Electrónicos             | 1,660,576,500.00 |
| Electrodomésticos        | 1,324,751,100.00 |
| Muebles                  |   757,661,000.00 |
| Instrumentos musicales   |   348,772,600.00 |
| Deportes y diversión     |   142,977,700.00 |


In [10]:
# 4. Top 5 Productos Globales
product_counts_global = df_concentrado['Producto'].value_counts()
productos_mas_vendidos_global = product_counts_global.nlargest(5).reset_index()
productos_mas_vendidos_global.columns = ['Producto', 'Cantidad Vendida']
productos_menos_vendidos_global = product_counts_global.nsmallest(5).reset_index()
productos_menos_vendidos_global.columns = ['Producto', 'Cantidad Vendida']

print("\n4. Top 5 Productos Más Vendidos Globalmente:")
print(productos_mas_vendidos_global.to_markdown(index=False))
print("\nTop 5 Productos Menos Vendidos Globalmente:")
print(productos_menos_vendidos_global.to_markdown(index=False))


4. Top 5 Productos Más Vendidos Globalmente:
| Producto                  |   Cantidad Vendida |
|:--------------------------|-------------------:|
| Mesa de noche             |                210 |
| Carrito de control remoto |                206 |
| Microondas                |                206 |
| Batería                   |                203 |
| Cama king                 |                201 |

Top 5 Productos Menos Vendidos Globalmente:
| Producto                    |   Cantidad Vendida |
|:----------------------------|-------------------:|
| Celular ABXY                |                157 |
| Auriculares con micrófono   |                158 |
| Mochila                     |                163 |
| Guitarra eléctrica          |                165 |
| Ciencia de datos con Python |                166 |
